In [0]:
catalog = "demos2025"
db = "recon"
table_name = "invoice_data"

spark.sql(f"DROP TABLE IF EXISTS {catalog}.{db}.{table_name}")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {catalog}.{db}.{table_name} (
    data STRING,
    n_operacao STRING,
    n_contrato STRING,
    tipo STRING,
    iof STRING,
    ir STRING,
    tarifa DOUBLE,
    natureza STRING,
    valor STRING
)
"""

spark.sql(create_table_query)

In [0]:
# Configurações do Auto Loader
schema = "data STRING, n_operacao STRING, n_contrato STRING, tipo STRING, iof STRING, ir STRING, tarifa DOUBLE, natureza STRING, valor STRING"

# Caminho do diretório de origem
source_path = "/Volumes/demos2025/recon/source_systems/csv/travelexbank/*.csv"

# Caminho do diretório de checkpoint
from datetime import datetime

# checkpoint_path = f"/Volumes/demos2025/recon/source_systems/csv/travelexbank/checkpoints/{datetime.now().strftime('%Y%m%d_%H%M%S')}/"
checkpoint_path = f"/Volumes/demos2025/recon/source_systems/csv/travelexbank/checkpoints/{table_name}/"

# Opções do Auto Loader para encoding e delimitador
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("delimiter", ";")              # Delimitador personalizado (;)
      .option("encoding", "ISO-8859-1")      # Codificação para caracteres especiais (ex: português)
      .option("header", "true")              # Primeira linha como cabeçalho
    #   .option("inferSchema", "true")         # Inferir schema automaticamente
      .option("cloudFiles.schemaLocation", checkpoint_path)  # Evolução de schema
      .schema(schema)               
      .load(source_path)
      .writeStream 
      .format("delta") 
      .outputMode("append") 
      .option("checkpointLocation", checkpoint_path) 
      .trigger(availableNow=True) 
      .toTable(f"{catalog}.{db}.{table_name}")
)